### Always Printing out the Current Version of Python This Notebook Runs

In [1]:
import sys
print("Python Version is: " + sys.version)

Python Version is: 3.11.2 | packaged by Anaconda, Inc. | (main, Mar 27 2023, 23:35:04) [MSC v.1916 64 bit (AMD64)]


### Importing Libraries and Changing Settings

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#sns.set(color_codes = True)
#sns.set(style="whitegrid")
#sns.set_palette("Set3")

# Get multiple outputs in the same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Ignore all warnings
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
pd.set_option('display.max_columns', None)

### Importing Cleaned & normalized CSV dataset

In [3]:
# For cluster analysis
df = pd.read_csv("Cleaned DF/_Normalized_XGBoost_DF.csv") 

FileNotFoundError: [Errno 2] No such file or directory: 'Cleaned DF/_Normalized_XGBoost_DF.csv'

### Creating a new dataframe from a copy of the original but only with ID and Satisfaction columns

In [ ]:
selected_columns = ['ID', 'Satisfaction']
new_df = df[selected_columns].copy()

### Importing the model from the train/test split on my other notebook

In [ ]:
import joblib

# Load the model from the file
model = joblib.load('Saved_Models\XGBoost_model.pkl')

### Running the model againts the entire dataset, no need to tweak anything, that was done in other notebook

In [ ]:
# Assuming you have already loaded the model as 'model' and have the dataset as 'df'

# Drop any unnecessary columns if needed
columns_to_drop = ['ID','Satisfaction']
df = df.drop(columns=columns_to_drop)

# Make predictions on the dataset
predictions = model.predict(df)

# Add the predictions to the DataFrame
df['Predicted_Target'] = predictions

In [ ]:
df

When you load a trained model, the model object itself doesn't inherently know what the target variable is. The model object primarily contains the learned parameters and configurations from the training process. Therefore, it's important to keep track of the target variable separately and ensure consistency when using the model.

During the training phase, you would have explicitly specified the target variable when training the model. The model is trained to learn the relationship between the input features and the specified target variable. The trained model captures this relationship in its learned parameters.

When you load the trained model into a new session or environment, you need to ensure that you have the necessary data with the same structure and feature names as used during training. You would typically provide the input features to the loaded model and expect the model to make predictions or classifications based on the learned patterns.

In summary, the loaded model itself doesn't have explicit knowledge of the target variable. It relies on you to provide the appropriate input features and expects you to know the target variable for making predictions or classifications.

### Adding the new_df of ID and Satisifaction columns to the df with predictions

In [ ]:
df2 = pd.concat([new_df,df], axis=1)
df2.head()

### Adding the prediction details to see what matches

In [ ]:
df2['Comparison'] = np.where(df2['Satisfaction'] == df2['Predicted_Target'], 'Match', 'Non-Matching')

In [ ]:
df2.head()

### Reorganizing the columns

In [ ]:
columns = list(df2.columns)
predicted_target_index = columns.index('Predicted_Target')
comparison_index = columns.index('Comparison')

columns.insert(2, columns.pop(predicted_target_index))
columns.insert(3, columns.pop(comparison_index))

df2 = df2[columns]

In [ ]:
df2.head()

### Viewing the matrix 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get the actual and predicted labels
y_actual = df2['Satisfaction']
y_predicted = df2['Predicted_Target']

# Create the confusion matrix
confusion_matrix = np.zeros((2, 2))

for actual, predicted in zip(y_actual, y_predicted):
    confusion_matrix[actual, predicted] += 1

# Set the labels for the matrix
labels = ['Negative', 'Positive']

# Plot the confusion matrix
fig, ax = plt.subplots()
im = ax.imshow(confusion_matrix, cmap='Blues')

# Add colorbar
#cbar = ax.figure.colorbar(im, ax=ax)

# Set tick labels
ax.set_xticks(np.arange(2))
ax.set_yticks(np.arange(2))
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)

# Set axis labels and title
ax.set_xlabel('Predicted Target')
ax.set_ylabel('Actual Satisfaction')
ax.set_title('Confusion Matrix')

# Loop over data dimensions and create text annotations
for i in range(2):
    for j in range(2):
        text = ax.text(j, i, int(confusion_matrix[i, j]),
                       ha="center", va="center", color="black")


# Display the plot
plt.show()


### Full Model Prediction

In [ ]:
df2.to_csv('Cleaned DF\Full_Model_Prediction_XGBoost_DF.csv', index=False)

In [ ]:
df2.head()

### Next is to add this prediction to the full and cleaned non-normalized dataframe

In [ ]:
# For cluster analysis

df3 = pd.read_csv("Cleaned DF/Cleaned_XGBoost_DF.csv") 

In [ ]:
df3.head()

In [ ]:
# Keep only these two columns from the dataframe with the predictions/results

columns_to_keep = df2[['Predicted_Target','Comparison']]
columns_to_keep.head()

In [ ]:
# Join them togethe (non-normalized data)

df4 = pd.concat([columns_to_keep,df3], axis=1)
df4.head()

In [ ]:
# Reorganize the columns

cols = df4.columns.tolist()  # Get the list of column names
cols = [cols[2]] + [cols[21]] + cols[:2] + cols[3:21] + cols[22:]  # Rearrange the column order
df4 = df4[cols]  # Reassign the columns to the DataFrame

In [ ]:
df4.head()

### Full Model Prediction

In [ ]:
df4.to_csv('Cleaned DF\Full_Model_Prediction_OG_XGBoost_DF.csv', index=False)

### Done